### Config file generator
This simple notebook provides a tool for generating map2loop config hjson files and a minimum python script, one of the more annoying parts of the process.
   
1. It first gets the project name and destination projection system (as EPSG number)   
   
2. It then asks for the paths to the different shape files (for WFS you will have to edit this manually afterwards)   
   
3. Next for each input file, you have to define which fields to use, and which keywords to look for in some of these fields 

4. Finally you can alter the user definable parameters
     
5. This generates a minimum python script for your data with all the paths and parameters hopefully correctly configured.This can be run as is or pasted into a new notebook. 


In [1]:
#!conda install ipyfilechooser
from ipyfilechooser import FileChooser
import geopandas as gpd
import ipywidgets as widgets
import pandas as pd
import os
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))
    
pd.options.display.max_columns = None
style = {'description_width': 'initial'}
c_l= {
#Orientations
  "d": "dip",                  #field that contains dip information
  "dd": "azimuth",             #field that contains dip direction information
  "sf": 'codedescpt',             #field that contains information on type of structure
  "bedding": 'bedding',            #text to search for in field defined by sf code to show that this is a bedding measurement
  "otype": 'dip direction',            #flag to determine measurement convention (currently 'strike' or 'dip direction')
  "bo": "deform",             #field that contains type of foliation
  "btype": 'BEOI',            #text to search for in field defined by bo code to show that this is an overturned bedding measurement
#stratigraphy
  "g": 'sub_provin',               #field that contains coarser stratigraphic coding
  "g2": 'province',              #field that contains alternate coarser stratigraphic coding if 'g' is blank
  "c": 'unit_name',                 #field that contains finer stratigraphic coding
  "ds": 'deposition',           #field that contains information about lithology
  "u": 'nsw_code',             #field that contains alternate stratigraphic coding (not used??)
  "r1": 'class',           #field that contains  extra lithology information
  "r2": 'igneous_ty',           #field that contains even more lithology information
  "sill": 'sill',              #text to search for in field defined by ds code to show that this is a sill
  "intrusive": 'intrusive',    #text to search for in field defined by r1 code to show that this is an intrusion
  "volcanic": 'volcanic',      #text to search for in field defined by ds code to show that this is an volcanic (not intrusion)
#mineral deposits
  "msc": 'SITE_CODE',          #field that contains site code of deposit
  "msn": 'SHORT_NAME',         #field that contains short name of deposit
  "mst": 'SITE_TYPE_',         #field that contains site type of deposit
  "mtc": 'TARGET_COM',         #field that contains target commodity of deposit
  "mscm": 'SITE_COMMO',        #field that contains site commodity of deposit
  "mcom": 'COMMODITY_',        #field that contains commodity group of deposit
  "minf": 'Infrastructure',    #text to search for in field defined by mst code that shows site to ignore
#timing
  "min": 'top_end_ag',         #field that contains minimum age of unit defined by ccode
  "max": 'base_start',         #field that contains maximum age of unit defined by ccode
#faults and folds
  "f": 'boundaryty',              #field that contains information on type of structure
  "fault": 'Fault',            #text to search for in field defined by f code to show that this is a fault
  "ff": 'codedescpt',  # field that contains information on type of structure
  "fold": 'cline',  #text to search for in field defined by ff code to show that this is a fold axial trace
  "fdip": 'dip',               # field for numeric fault dip value
  "fdipnull": '0',         # text to search for in field defined by fdip to show that this has no known dip
  "fdipdir": 'faultdipdi',        # field for text fault dip direction value 
  "fdipdir_flag": 'alpha',        # flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast    
  "fdipest": 'faultdipan',        # field for text fault dip estimate value
  "fdipest_vals": 'Moderate,Listric,Steep,Vertical',        # text to search for in field defined by fdipest to give fault dip estimate in increasing steepness
  "n": 'name',                 #field that contains information on name of fault (not used??)
  "t": 'codedescpt',                 #field that contains information on type of fold
  "syn": 'Syncline',           #text to search for in field defined by t to show that this is a syncline
#ids
  "o": 'unique_id',             #field that contains unique id of geometry object
  "gi": 'objectid',            #field that contains unique id of structure point
  "deposit_dist":500           # dist between mindep and contact to be considered "on contact"
}

In [2]:
test_data_name_choice=widgets.Text(
    placeholder='Test',
    value='Test',
    description='Project Name:',
    disabled=False,
    style=style,
    tooltip='Name of directory to hold config file (relative path)'
)
display(test_data_name_choice)

epsg=pd.read_csv('source_data/epsg.csv')

crs_choice=widgets.Dropdown(
    options=epsg['epsg'],
    description='CRS EPSG:',
    disabled=False
)
display(crs_choice)


Text(value='Test', description='Project Name:', placeholder='Test', style=DescriptionStyle(description_width='…

Dropdown(description='CRS EPSG:', options=('28348: GDA94 // MGA zone 48', '28349: GDA94 // MGA zone 49', '2835…

In [3]:
test_data_name=test_data_name_choice.value
epsg_code=crs_choice.value.split(":")
dst_crs='epsg:'+epsg_code[0]
printmd('**'+test_data_name+", "+dst_crs+'**')

**Paterson_1, epsg:28351**

In [4]:
minx_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min x:',
    disabled=False,
    style=style
)
display(minx_choice)

maxx_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max x:',
    disabled=False,
    style=style
)
display(maxx_choice)

miny_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min y:',
    disabled=False,
    style=style
)
display(miny_choice)

maxy_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max y:',
    disabled=False,
    style=style
)
display(maxy_choice)

minz_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min z:',
    disabled=False,
    style=style
)
display(minz_choice)

maxz_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max z:',
    disabled=False,
    style=style
)
display(maxz_choice)


Text(value='0', description='min x:', placeholder='0', style=DescriptionStyle(description_width='initial'))

Text(value='1', description='max x:', placeholder='1', style=DescriptionStyle(description_width='initial'))

Text(value='0', description='min y:', placeholder='0', style=DescriptionStyle(description_width='initial'))

Text(value='1', description='max y:', placeholder='1', style=DescriptionStyle(description_width='initial'))

Text(value='0', description='min z:', placeholder='0', style=DescriptionStyle(description_width='initial'))

Text(value='1', description='max z:', placeholder='1', style=DescriptionStyle(description_width='initial'))

In [5]:
minx=minx_choice.value
maxx=maxx_choice.value
miny=miny_choice.value
maxy=maxy_choice.value
minz=minz_choice.value
maxz=maxz_choice.value


structure_file_m = FileChooser('../')
structure_file_m.title = '<b>Choose Bedding Orientation point file</b>'
display(structure_file_m)

fault_file_m = FileChooser('../')
fault_file_m.title = '<b>Choose Fault polyline file</b>'
display(fault_file_m)

fold_file_m = FileChooser('../')
fold_file_m.title = '<b>Choose Fold polyline file</b>'
display(fold_file_m)

geology_file_m = FileChooser('../')
geology_file_m.title = '<b>Choose Geology polygon file</b>'
display(geology_file_m)

mindep_file_m = FileChooser('../')
mindep_file_m.title = '<b>Choose Mineral Deposit point file (or use null_mindeps.shp from source_data directory)</b>'
#mindep_file_m.default_path = '../source_data/'
mindep_file_m.default_filename = 'null_mindeps.shp'
display(mindep_file_m)



FileChooser(path='../', filename='', title='HTML(value='<b>Choose Bedding Orientation point file</b>')', show_…

FileChooser(path='../', filename='', title='HTML(value='<b>Choose Fault polyline file</b>')', show_hidden='Fal…

FileChooser(path='../', filename='', title='HTML(value='<b>Choose Fold polyline file</b>')', show_hidden='Fals…

FileChooser(path='../', filename='', title='HTML(value='<b>Choose Geology polygon file</b>')', show_hidden='Fa…

FileChooser(path='../', filename='null_mindeps.shp', title='HTML(value='<b>Choose Mineral Deposit point file (…

In [6]:
if(str(structure_file_m.selected_path) =='None'):
    structure_file=''
else:
    structure_file=structure_file_m.selected_path.replace('\\','/')+'/'+structure_file_m.selected_filename
if(str(fault_file_m.selected_path) =='None'):
    fault_file=''
else:
    fault_file=fault_file_m.selected_path.replace('\\','/')+'/'+fault_file_m.selected_filename
if(str(fold_file_m.selected_path) =='None'):
    fold_file=''
else:
    fold_file=fold_file_m.selected_path.replace('\\','/')+'/'+fold_file_m.selected_filename
if(str(geology_file_m.selected_path) =='None'):
    geology_file=''
else:
    geology_file=geology_file_m.selected_path.replace('\\','/')+'/'+geology_file_m.selected_filename
if(str(mindep_file_m.selected_path) =='None'):
    mindep_file=''
else:
    mindep_file=mindep_file_m.selected_path.replace('\\','/')+'/'+mindep_file_m.selected_filename

display(structure_file,fault_file,fold_file,geology_file,mindep_file)

'C:/Users/00077806/Dropbox/Loop_paterson/source_data/Layering dip and dip direction.shp'

'C:/Users/00077806/Dropbox/Loop_paterson/source_data/Faults_Polygons.shp'

'C:/Users/00077806/Dropbox/Loop_paterson/source_data/Faults_Polygons.shp'

'C:/Users/00077806/Dropbox/Loop_paterson/source_data/Paterson_Polygons_4.shp'

'C:/Users/00077806/Dropbox/Loop_paterson/map2loop2-notebooks/source_data/null_mindeps.shp'

In [7]:
if(not structure_file==''):
    structure=gpd.read_file(structure_file)
    display(structure.head())
    
    cols=['No_col']
    for sc in structure.columns:
        cols.append(sc)
        
    printmd('**Based on file: '+structure_file+'**')
    printmd('\n**Field that contains unique id of structure point**')
    gi_choice=widgets.Dropdown(
        options=cols,
        description='Unique ID',
        disabled=False

    )
    display(gi_choice)

    printmd('**Field that contains information on type of structure (e.g. bedding, cleavage etc.)**')

    sf_choice=widgets.Dropdown(
        options=cols,
        description='Structure',
        disabled=False
    )
    display(sf_choice)

    printmd('**Text to search for in field defined by type of structure code to show that this is a bedding measurement**')

    bedding_choice=widgets.Text(
        placeholder='Bed',
        value='Bed',
        description='Bedding text',
        disabled=False
    )
    display(bedding_choice)

    printmd('**Field that contains dip information**')

    d_choice=widgets.Dropdown(
        options=cols,
        description='Dip',
        disabled=False
    )
    display(d_choice)

    printmd('**Field that contains dip direction or strike information**')

    dd_choice=widgets.Dropdown(
        options=cols,
        description='Dip Direction',
        disabled=False
    )
    display(dd_choice)

    printmd('**Flag to determine measurement convention**')

    otype_choice=widgets.Dropdown(
        options=('dip direction','strike'),
        description='Convention',
        disabled=False
    )
    display(otype_choice)

    printmd('**Field that contains polarity of foliation (upward facing or not)**')

    bo_choice=widgets.Dropdown(
        options=cols,
        description='Polarity',
        disabled=False
    )
    display(bo_choice)

    printmd('**Text to search for in field defined by Polarity code to show that this is an overturned bedding measurement**')

    btype_choice=widgets.Text(
        placeholder='Blah',
        value='Blah',
        description='Pol. text',
        disabled=False
    )
    display(btype_choice)



,Id,layer_type,dip,dip_direct,dip_num,dip_dir_nu,geometry
0,0,bedding,46,355,46,355,POINT (123.25703 -23.04261)
1,0,bedding,70,175,70,175,POINT (123.24056 -23.02927)
2,0,bedding,15,120,15,120,POINT (123.30823 -23.07090)
3,0,bedding,20,180,20,180,POINT (123.31403 -23.21146)
4,0,bedding,20,045,20,45,POINT (123.28077 -23.24109)


**Based on file: C:/Users/00077806/Dropbox/Loop_paterson/source_data/Layering dip and dip direction.shp**


**Field that contains unique id of structure point**

Dropdown(description='Unique ID', options=('No_col', 'Id', 'layer_type', 'dip', 'dip_direct', 'dip_num', 'dip_…

**Field that contains information on type of structure (e.g. bedding, cleavage etc.)**

...

Dropdown(description='Convention', options=('dip direction', 'strike'), value='dip direction')

**Field that contains polarity of foliation (upward facing or not)**

Dropdown(description='Polarity', options=('No_col', 'Id', 'layer_type', 'dip', 'dip_direct', 'dip_num', 'dip_d…

**Text to search for in field defined by Polarity code to show that this is an overturned bedding measurement**

Text(value='Blah', description='Pol. text', placeholder='Blah')

In [8]:
c_l['gi']=gi_choice.value
c_l['sf']=sf_choice.value
c_l['bedding']=bedding_choice.value
c_l['d']=d_choice.value
c_l['dd']=dd_choice.value
c_l['otype']=otype_choice.value
c_l['bo']=bo_choice.value
c_l['btype']=btype_choice.value

if(not fault_file==''):
    fault=gpd.read_file(fault_file)
    display(fault.head())
    printmd('**Based on file: '+fault_file+'**')

    cols=['No_col']
    for sc in fault.columns:
        cols.append(sc)
        
    printmd('\n**Field that contains information on type of structure (fault, fold axial trace etc.)**')


    f_choice=widgets.Dropdown(
        options=cols,
        description='Fault field:',
        disabled=False
    )
    display(f_choice)

    printmd('**Text to search for in field defined by Fault field code to show that this is a fault**')

    fault_choice=widgets.Text(
        placeholder='Fault',
        value='Fault',
        description='Fault text:',
        disabled=False
    )
    display(fault_choice)

    printmd('**Field for numeric fault dip value**')

    fdip_choice=widgets.Dropdown(
        options=cols,
        description='Dip:',
        disabled=False
    )
    display(fdip_choice)

    printmd('**Text to search for in field defined by Dip to show that this has no known dip**')

    fdipnull_choice=widgets.Text(
        placeholder='0',
        value='0',
        description='NULL dip:',
        disabled=False
    )
    display(fdipnull_choice)

    printmd('**Field for text fault dip direction value (southeast etc.)**')

    fdipdir_choice=widgets.Dropdown(
        options=cols,
        description='Dip Direction:',
        disabled=False
    )
    display(fdipdir_choice)

    printmd('**Flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast**')

    fdipdir_flag_choice=widgets.Dropdown(
        options=('alpha','num'),
        description='Dip Convention:',
        disabled=False,
        style=style
    )
    display(fdipdir_flag_choice)

    printmd('**Field for text fault dip estimate value (Moderate, Steep etc.)**')

    fdipest_choice=widgets.Dropdown(
        options=cols,
        description='Dip estimate:',
        disabled=False
    )
    display(fdipest_choice)

,Id,Type,Interp_fro,Comments,Dip_est,Dip_dir,Plunge_est,Plunge_dir,Max_age_ev,Min_age_ev,Name,geometry
0,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (420386.400 7655912.409, 419910.149..."
1,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (421516.702 7655474.258, 420735.651..."
2,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (424778.067 7650175.351, 423222.626..."
3,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (423342.275 7652872.900, 421852.096..."
4,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (411574.194 7642187.068, 411675.194..."


**Based on file: C:/Users/00077806/Dropbox/Loop_paterson/source_data/Faults_Polygons.shp**


**Field that contains information on type of structure (fault, fold axial trace etc.)**

Dropdown(description='Fault field:', options=('No_col', 'Id', 'Type', 'Interp_fro', 'Comments', 'Dip_est', 'Di…

**Text to search for in field defined by Fault field code to show that this is a fault**

...

Dropdown(description='Dip Direction:', options=('No_col', 'Id', 'Type', 'Interp_fro', 'Comments', 'Dip_est', '…

**Flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast**

Dropdown(description='Dip Convention:', options=('alpha', 'num'), style=DescriptionStyle(description_width='in…

**Field for text fault dip estimate value (Moderate, Steep etc.)**

Dropdown(description='Dip estimate:', options=('No_col', 'Id', 'Type', 'Interp_fro', 'Comments', 'Dip_est', 'D…

In [9]:
printmd('**Text to search for in field defined by fdipest to give fault dip estimate in increasing steepness**')

if(fdipest_choice.value=='No_col'):
    c_l['fdipest_vals']='No_col'
else:
    fdipest_vals_choice=widgets.Text(
        placeholder='Sorted dip order (shallow to steep)',
        value=str(fault[fdipest_choice.value].unique()).replace("[","").replace("]","").replace(" ",","),
        description='Text Dips:',
        disabled=False
    )
    #display(fdipest_vals)
    c_l['fdipest_vals']=fdipest_vals_choice.value


    
printmd('**Field that contains information on name of fault (not used??)**')

n_choice=widgets.Dropdown(
    options=cols,
    description='Name:',
    disabled=False
)
display(n_choice)

**Text to search for in field defined by fdipest to give fault dip estimate in increasing steepness**

**Field that contains information on name of fault (not used??)**

Dropdown(description='Name:', options=('No_col', 'Id', 'Type', 'Interp_fro', 'Comments', 'Dip_est', 'Dip_dir',…

In [10]:
c_l['f']=f_choice.value
c_l['fault']=fault_choice.value
c_l['fdip']=fdip_choice.value
c_l['fdipnull']=fdipnull_choice.value
c_l['fdipdir']=fdipdir_choice.value
c_l['fdipdir_flag']=fdipdir_flag_choice.value
c_l['fdipest']=fdipest_choice.value
c_l['n']=n_choice.value

if(not fold_file==''):
    fold=gpd.read_file(fold_file)
    display(fold.head())
    cols=['No_col']
    for sc in fold.columns:
        cols.append(sc)
        
    printmd('**Based on file:'+' '+fold_file+'**')

    printmd('\n**Field that contains information on type of structure (fold, fault etc.)**')

    ff_choice=widgets.Dropdown(
        options=cols,
        description='Fold field:',
        disabled=False
    )
    display(ff_choice)

    printmd('**Text to search for in field defined by Fold field to show that this is a fold axial trace**')

    fold_choice=widgets.Text(
        placeholder='Fold',
        value='Fold',
        description='Fold text:',
        disabled=False
    )
    display(fold_choice)

    printmd('**Field that contains information on type of fold (anticline, syncline)**')

    t_choice=widgets.Dropdown(
        options=cols,
        description='Polarity field:',
        disabled=False
    )
    display(t_choice)

    printmd('**Text to search for in field defined by Polarity Field to show that this is a syncline**')

    syn_choice=widgets.Text(
        placeholder='syn',
        value='syn',
        description='Syncline text:',
        disabled=False
    )
    display(syn_choice)


,Id,Type,Interp_fro,Comments,Dip_est,Dip_dir,Plunge_est,Plunge_dir,Max_age_ev,Min_age_ev,Name,geometry
0,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (420386.400 7655912.409, 419910.149..."
1,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (421516.702 7655474.258, 420735.651..."
2,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (424778.067 7650175.351, 423222.626..."
3,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (423342.275 7652872.900, 421852.096..."
4,0,Fault - undivided,None,None,None,None,None,None,None,None,None,"LINESTRING (411574.194 7642187.068, 411675.194..."


**Based on file: C:/Users/00077806/Dropbox/Loop_paterson/source_data/Faults_Polygons.shp**


**Field that contains information on type of structure (fold, fault etc.)**

Dropdown(description='Fold field:', options=('No_col', 'Id', 'Type', 'Interp_fro', 'Comments', 'Dip_est', 'Dip…

**Text to search for in field defined by Fold field to show that this is a fold axial trace**

Text(value='Fold', description='Fold text:', placeholder='Fold')

**Field that contains information on type of fold (anticline, syncline)**

Dropdown(description='Polarity field:', options=('No_col', 'Id', 'Type', 'Interp_fro', 'Comments', 'Dip_est', …

**Text to search for in field defined by Polarity Field to show that this is a syncline**

Text(value='syn', description='Syncline text:', placeholder='syn')

In [12]:
geology_file='C:/Users/00077806/Dropbox/Loop_paterson/source_data/Paterson_Polygons_4WG.shp'

In [13]:
c_l['ff']=ff_choice.value
c_l['fold']=fold_choice.value
c_l['t']=t_choice.value
c_l['syn']=syn_choice.value

if(not geology_file==''):
    geology=gpd.read_file(geology_file)
    display(geology.head())
    cols=['No_col']
    for sc in geology.columns:
        cols.append(sc)
    printmd('**Based on file:'+' '+geology_file+'**')

    printmd('\n**Field that contains unique ID**')

    o_choice=widgets.Dropdown(
        options=cols,
        description='Unique ID',
        disabled=False
    )
    display(o_choice)

    printmd('**Field that contains finer stratigraphic coding**')

    c_choice=widgets.Dropdown(
        options=cols,
        description='Formation',
        disabled=False
    )
    display(c_choice)

    printmd('\n**Field that contains coarser stratigraphic coding**')

    g_choice=widgets.Dropdown(
        options=cols,
        description='Group',
        disabled=False
    )
    display(g_choice)

    printmd('**Field that contains alternate coarser stratigraphic coding if Group is blank**')

    g2_choice=widgets.Dropdown(
        options=cols,
        description='Group alt',
        disabled=False
    )
    display(g2_choice)


    printmd('**Field that contains alternate stratigraphic coding (used to check for "drift" formations'')**')

    u_choice=widgets.Dropdown(
        options=cols,
        description='Strat alt',
        disabled=False
    )
    display(u_choice)

    printmd('**Field that contains information about lithology**')

    ds_choice=widgets.Dropdown(
        options=cols,
        description='Description:',
        disabled=False
    )
    display(ds_choice)

    printmd('**Field that contains extra lithology information**')

    r1_choice=widgets.Dropdown(
        options=cols,
        description='Litho',
        disabled=False
    )
    display(r1_choice)

    printmd('**Field that contains even more lithology information**')

    r2_choice=widgets.Dropdown(
        options=cols,
        description='Litho alt',
        disabled=False
    )
    display(r2_choice)

    printmd('**Text to search for in field defined by Description code to show that this is a sill**')

    sill_choice=widgets.Text(
        placeholder='sill',
        value='sill',
        description='Sill text',
        disabled=False
    )
    display(sill_choice)

    printmd('**Text to search for in field defined by Litho code to show that this is an intrusion**')

    intrusive_choice=widgets.Text(
        placeholder='intrusive',
        value='intrusive',
        description='Intrusive text',
        disabled=False
    )
    display(intrusive_choice)

    printmd('**Text to search for in field defined by Description code to show that this is an volcanic (not intrusion)**')


    volcanic_choice=widgets.Text(
        placeholder='volc',
        value='volc',
        description='Volc text',
        disabled=False
    )
    display(volcanic_choice)

    printmd('**Field that contains minimum age of unit defined by Formation**')

    min_choice=widgets.Dropdown(
        options=cols,
        description='Min age',
        disabled=False
    )
    display(min_choice)

    printmd('**Field that contains maximum age of unit defined by Formation**')

    max_choice=widgets.Dropdown(
        options=cols,
        description='Max age',
        disabled=False
    )
    display(max_choice)



,Id,Formation,Interp_fro,Comments,LITHSTRTNO,TECTNO,Formatio_1,UNITNAME,CODE,TECTNAME,TECTCODE,Legend_nar,Lithology,ROCKTYPE1,LITHNAME1,ROCKTYPE2,LITHNAME2,Maximum_ag,Minimum_ag,Rank,GSWA_statu,Parent_uni,TOL_Tecton,SUBBASIN,BASIN,SUPERBASIN,Formatio_2,Relative_m,Relative_1,geometry
0,0,Browne Formation,GSWA Rpt 93; mag,"Salt tectonics in Browne Formation, which is e...",None,None,Browne Formation,Browne Formation,None,None,None,"Dolomite, massive, bedded and vein-type evapor...",None,sedimentary siliciclastic,siliciclastic + lesser carbonate,None,None,850.0,720.0,Formation,Formal,Buldya Group,Centralian Superbasin,None,Officer Basin,Centralian Superbasin,Browne Formation,840,810,"POLYGON ((517764.938 7441151.216, 519061.399 7..."
1,0,Broadhurst Formation,None,None,1834,None,Broadhurst Formation,Broadhurst Formation,P_-THb-shh,None,None,"Carbonaceous shale and siltstone, pyritic and ...",None,sedimentary siliciclastic,shale,None,None,911.0,650.0,Formation,Formal,Throssell Range Group,Centralian Superbasin,None,Yeneena Basin,Centralian Superbasin,Broadhurst Formation,840,810,"POLYGON ((527698.905 7446034.264, 528032.517 7..."
2,0,Tabletop Terrane,GSWA 100,overlain by Throssell Range Gp,None,PTRT,Tabletop Terrane,None,None,Tabletop Terrane,PTRT,igneous and metamorphic rocks,igneous and metamorphic rocks,None,None,None,None,2500.0,1291.0,Province terrane/zone,Formal,Rudall Province,Rudall Province,None,None,None,Tabletop Terrane,1300,1250,"POLYGON ((528503.578 7446369.544, 528956.207 7..."
3,0,Broadhurst Formation,None,None,1834,None,Broadhurst Formation,Broadhurst Formation,P_-THb-shh,None,None,"Carbonaceous shale and siltstone, pyritic and ...",None,sedimentary siliciclastic,shale,None,None,911.0,650.0,Formation,Formal,Throssell Range Group,Centralian Superbasin,None,Yeneena Basin,Centralian Superbasin,Broadhurst Formation,840,810,"POLYGON ((502727.350 7449565.504, 503173.835 7..."
4,0,Choorun Formation,GSWA 250; mag,None,3824,None,Choorun Formation,Choorun Formation,P_-TAc-sg,None,None,"Pebble to cobble conglomerate, pebbly sandston...",None,sedimentary siliciclastic,conglomerate + sandstone,None,None,1000.0,635.0,Formation,Formal,Tarcunyah Group,Centralian Superbasin,Gibson Sub-basin,Officer Basin,Centralian Superbasin,Choorun Formation,833,831,"POLYGON ((514002.626 7451641.298, 514113.621 7..."


**Based on file: C:/Users/00077806/Dropbox/Loop_paterson/source_data/Paterson_Polygons_4WG.shp**


**Field that contains unique ID**

Dropdown(description='Unique ID', options=('No_col', 'Id', 'Formation', 'Interp_fro', 'Comments', 'LITHSTRTNO'…

**Field that contains finer stratigraphic coding**

...

Text(value='volc', description='Volc text', placeholder='volc')

**Field that contains minimum age of unit defined by Formation**

Dropdown(description='Min age', options=('No_col', 'Id', 'Formation', 'Interp_fro', 'Comments', 'LITHSTRTNO', …

**Field that contains maximum age of unit defined by Formation**

Dropdown(description='Max age', options=('No_col', 'Id', 'Formation', 'Interp_fro', 'Comments', 'LITHSTRTNO', …

In [14]:
c_l['o']=o_choice.value
c_l['g']=g_choice.value
c_l['g2']=g2_choice.value
c_l['c']=c_choice.value
c_l['ds']=ds_choice.value
c_l['u']=u_choice.value
c_l['r1']=r1_choice.value
c_l['r2']=r2_choice.value
c_l['sill']=sill_choice.value
c_l['intrusive']=intrusive_choice.value
c_l['volcanic']=volcanic_choice.value
c_l['min']=min_choice.value
c_l['max']=max_choice.value

mindep=gpd.read_file(mindep_file)
display(mindep.head())
cols=['No_col']
for sc in mindep.columns:
    cols.append(sc)

printmd('**Based on file:'+' '+mindep_file+'**')

printmd('\n**Field that contains site code of deposit**')

msc_choice=widgets.Dropdown(
    options=cols,
    value='SITE_CODE',
    description='Site code',
    disabled=False
)
display(msc_choice)

printmd('**Field that contains short name of deposit (e.g. Jones deposit)**')

msn_choice=widgets.Dropdown(
    options=cols,
    value='SHORT_NAME',
    description='Short Name',
    disabled=False
)
display(msn_choice)

printmd('**Field that contains site type of deposit (Mine, occurrence etc.)**')

mst_choice=widgets.Dropdown(
    options=cols,
    value='SITE_TYPE_',
    description='Site type',
    disabled=False
)
display(mst_choice)


printmd('**Field that contains target commodity of deposit (Commodity)**')

mtc_choice=widgets.Dropdown(
    options=cols,
    value='TARGET_COM',
    description='Target Comm',
    disabled=False
)
display(mtc_choice)

printmd('**Field that contains site commodity of deposit (ore Mineral)**')

mscm_choice=widgets.Dropdown(
    options=cols,
    value='SITE_COMMO',
    description='Site Commodity',
    disabled=False
)
display(mscm_choice)

printmd('**Field that contains commodity group of deposit (e.g. Metal)**')

mcom_choice=widgets.Dropdown(
    options=cols,
    value='COMMODITY_',
    description='Commodity',
    disabled=False,
    style=style
)
display(mcom_choice)

printmd('**Text to search for in field defined by Site type code that shows site to ignore**')

minf_choice=widgets.Text(
    placeholder='Cons',
    value='Cons',
    description='Not mine text',
    disabled=False
)
display(minf_choice)


,SITE_CODE,SITE_TITLE,SHORT_NAME,SITE_COMMO,SITE_TYPE_,SITE_SUB_T,SITE_STAGE,TARGET_COM,COMMODITY_,PROJ_CODE,PROJ_TITLE,WEB_LINK,EXTRACT_DA,geometry
0,S0001482,Vivash Gorge,Vivash Gorge,Fe,Deposit,Unspecified,Undeveloped,IRON ORE,IRON,J03931,Brockman 4 Area / Rio Tinto,http://minedexext.dmp.wa.gov.au/minedex/extern...,2018-05-07,POINT (505671.537 7495706.968)
1,S0231563,Eliwana Exploration Camp,Exploration camp,Fe,Infrastructure,Town/Village/Camp,Operating,IRON ORE,IRON,J03436,Western Hub / FMG,http://minedexext.dmp.wa.gov.au/minedex/extern...,2018-05-07,POINT (503344.199 7511681.103)
2,S0233921,Eliwana Far East 2 Marra Mamba,Eliwana Far East 2 Marra Mamba,Fe,Deposit,Unspecified,Undeveloped,IRON ORE,IRON,J03436,Western Hub / FMG,http://minedexext.dmp.wa.gov.au/minedex/extern...,2018-05-07,POINT (505699.008 7509917.737)


**Based on file: C:/Users/00077806/Dropbox/Loop_paterson/map2loop2-notebooks/source_data/null_mindeps.shp**


**Field that contains site code of deposit**

Dropdown(description='Site code', index=1, options=('No_col', 'SITE_CODE', 'SITE_TITLE', 'SHORT_NAME', 'SITE_C…

**Field that contains short name of deposit (e.g. Jones deposit)**

...

Dropdown(description='Site Commodity', index=4, options=('No_col', 'SITE_CODE', 'SITE_TITLE', 'SHORT_NAME', 'S…

**Field that contains commodity group of deposit (e.g. Metal)**

Dropdown(description='Commodity', index=9, options=('No_col', 'SITE_CODE', 'SITE_TITLE', 'SHORT_NAME', 'SITE_C…

**Text to search for in field defined by Site type code that shows site to ignore**

Text(value='Cons', description='Not mine text', placeholder='Cons')

In [15]:
c_l['msc']=msc_choice.value
c_l['msn']=msn_choice.value
c_l['mst']=mst_choice.value
c_l['mtc']=mtc_choice.value
c_l['mscm']=mscm_choice.value
c_l['mcom']=mcom_choice.value
c_l['minf']=minf_choice.value

if(not os.path.isdir(test_data_name)):
    os.mkdir(test_data_name)
f=open(test_data_name+'/'+test_data_name+'_config.hjson',"w")
f.write(str(c_l).replace('\n',''))
f.write("\n")

f.close()

In [16]:
c_l

{'d': 'dip',
 'dd': 'dip_direct',
 'sf': 'layer_type',
 'bedding': 'bedding',
 'otype': 'dip direction',
 'bo': 'No_col',
 'btype': 'normal',
 'g': 'Parent_uni',
 'g2': 'TOL_Tecton',
 'c': 'Formation',
 'ds': 'ROCKTYPE1',
 'u': 'SUPERBASIN',
 'r1': 'LITHNAME1',
 'r2': 'Legend_nar',
 'sill': 'sill',
 'intrusive': 'granitic',
 'volcanic': 'volc',
 'msc': 'SITE_CODE',
 'msn': 'SHORT_NAME',
 'mst': 'SITE_TYPE_',
 'mtc': 'TARGET_COM',
 'mscm': 'SITE_COMMO',
 'mcom': 'COMMODITY_',
 'minf': 'Cons',
 'min': 'Minimum_ag',
 'max': 'Maximum_ag',
 'f': 'Type',
 'fault': 'Fault',
 'ff': 'Type',
 'fold': 'Fold',
 'fdip': 'Dip_est',
 'fdipnull': '-999',
 'fdipdir': 'Dip_dir',
 'fdipdir_flag': 'num',
 'fdipest': 'No_col',
 'fdipest_vals': 'No_col',
 'n': 'Id',
 't': 'No_col',
 'syn': 'syn',
 'o': 'Id',
 'gi': 'Id',
 'deposit_dist': 500}

In [72]:
printmd('**aus: Indicates if area is in Australia for using ASUD. Defaults to True.**')

aus_choice=widgets.Text(
    options=cols,
    value='True',
    description='aus',
    disabled=False
)
display(aus)

printmd('**deposits: Mineral deposit names for focused topology extraction. Defaults to "Fe,Cu,Au,NONE".**')

deposits_choice=widgets.Text(
    options=cols,
    value='"Fe,Cu,Au,NONE"',
    description='deposits',
    disabled=False
)
display(deposits)

printmd('**dtb: Path to depth to basement grid. Defaults to ''.**')

dtb_choice=widgets.Text(
    options=cols,
    value='',
    description='dtb',
    disabled=False
)
display(dtb)

printmd('**orientation_decimate: Save every nth orientation data point. Defaults to 0.**')

orientation_decimate_choice=widgets.Text(
    options=cols,
    value='0',
    description='orientation_decimate',
    disabled=False
)
display(orientation_decimate)

printmd('**contact_decimate: Save every nth contact data point. Defaults to 5.**')

contact_decimate_choice=widgets.Text(
    options=cols,
    value='5',
    description='contact_decimate',
    disabled=False
)
display(contact_decimate)

printmd('**intrusion_mode: 1 to exclude all intrusions from basal contacts, 0 to only exclude sills. Defaults to 0.**')

intrusion_mode_choice=widgets.Text(
    options=cols,
    value='0',
    description='intrusion_mode',
    disabled=False
)
display(intrusion_mode)

printmd('**interpolation_spacing: Interpolation grid spacing in meters. Defaults to 500.**')
interpolation_spacing_choice=widgets.Text(
    options=cols,
    value= '500',
    description='interpolation_spacing',
    disabled=False
)
display(interpolation_spacing)

printmd('**misorientation: maximimum misorientation between poles to bedding girdles to allow to be part of same supergroup. Defaults to 30.**')

misorientation_choice=widgets.Text(
    options=cols,
    value='30',
    description='misorientation',
    disabled=False
)
display(misorientation)

printmd('**interpolation_scheme: What interpolation function to use of scipy_rbf (radial basis) or scipy_idw (inverse distance weighted). Defaults to \'scipy_rbf\'.**')

interpolation_scheme_choice=widgets.Text(
    options=cols,
    value='scipy_rbf',
    description='interpolation_scheme',
    disabled=False
)
display(interpolation_scheme)

printmd('**fault_decimate: Save every nth fault data point. Defaults to 5.**')

fault_decimate_choice=widgets.Text(
    options=cols,
    value='5',
    description='fault_decimate',
    disabled=False
)
display(fault_decimate)

printmd('**min_fault_length: Min fault length to be considered. Defaults to 5000.**')

min_fault_length_choice=widgets.Text(
    options=cols,
    value='5000',
    description='min_fault_length',
    disabled=False
)
display(min_fault_length)

printmd('**fault_dip: Defaults to 90.**')

fault_dip_choice=widgets.Text(
    options=cols,
    value='90',
    description='fault_dip',
    disabled=False
)
display(fault_dip)

printmd('**pluton_dip: Dip of contacts. Defaults to 45.**')

pluton_dip_choice=widgets.Text(
    options=cols,
    value='45',
    description='pluton_dip',
    disabled=False
)
display(pluton_dip)

printmd('**pluton_form: Possible forms from domes, saucers or pendant. Defaults to \'domes\'.**')

pluton_form_choice=widgets.Text(
    options=cols,
    value='domes',
    description='pluton_form',
    disabled=False
)
display(pluton_form)

printmd('**dist_buffer: Buffer for processing plutons for faults that do not quite touch pluton. Defaults to 10.**')

dist_buffer_choice=widgets.Text(
    options=cols,
    value='10',
    description='dist_buffer',
    disabled=False
)
display(dist_buffer)

printmd('**contact_dip: Assign fixed dip to all contacts, -99 means interpolated local orientaiton field is used. Defaults to -999.**')

contact_dip_choice=widgets.Text(
    options=cols,
    value='-999',
    description='contact_dip',
    disabled=False
)
display(contact_dip)

printmd('**contact_orientation_decimate: Save every nth contact orientation point. Defaults to 5.**')

contact_orientation_decimate_choice=widgets.Text(
    options=cols,
    value='5',
    description='contact_orientation_decimate',
    disabled=False
)
display(contact_orientation_decimate)

printmd('**null_scheme: How null values present in the dtb. Defaults to \'null\'.**')

null_scheme_choice=widgets.Text(
    options=cols,
    value='null',
    description='null_scheme',
    disabled=False
)
display(null_scheme)

printmd('**thickness_buffer: Maximum distance to search for next contact when calculating formation thickness. Defaults to 5000.**')

thickness_buffer_choice=widgets.Text(
    options=cols,
    value='5000',
    description='thickness_buffer',
    disabled=False
)
display(thickness_buffer)

printmd('**max_thickness_allowed: Maximum formation thickness allowed for calculation. Defaults to 10000.**')

max_thickness_allowed_choice=widgets.Text(
    options=cols,
    value='10000',
    description='max_thickness_allowed',
    disabled=False
)
display(max_thickness_allowed)

printmd('**fold_decimate: Save every nth fold data point. Defaults to 5.**')

fold_decimate_choice=widgets.Text(
    options=cols,
    value='5',
    description='fold_decimate',
    disabled=False
)
display(fold_decimate)

printmd('**fat_step: How much to step out normal to the fold axial trace for limb orientations. Defaults to 750.**')

fat_step_choice=widgets.Text(
    options=cols,
    value='750',
    description='fat_step',
    disabled=False
)
display(fat_step)

printmd('**close_dip: Dip to assign to all new orientations. Defaults to -999.**')

close_dip_choice=widgets.Text(
    options=cols,
    value='-999',
    description='close_dip',
    disabled=False
)
display(close_dip)

printmd('**use_interpolations: Defaults to False.**')

use_interpolations_choice=widgets.Text(
    options=cols,
    value='False',
    description='use_interpolations',
    disabled=False
)
display(use_interpolations)

printmd('**use_fat: Use fake limb orientation data around fold hinges Defaults to False.**')

use_fat_choice=widgets.Text(
    options=cols,
    value='False',
    description='use_fat',
    disabled=False
)
display(use_fat)

printmd('**clut_path: User defined look up table for formation colours Default \'\'.**')

clut_path_choice=widgets.Text(
    options=cols,
    value='',
    description='clut_path',
    disabled=False
)
display(clut_path)





**aus: Indicates if area is in Australia for using ASUD. Defaults to True.**

Text(value='True', description='aus')

**deposits: Mineral deposit names for focused topology extraction. Defaults to "Fe,Cu,Au,NONE".**

Text(value='"Fe,Cu,Au,NONE"', description='deposits')

**dtb: Path to depth to basement grid. Defaults to .**

...

Text(value='False', description='use_interpolations')

**use_fat: Use fake limb orientation data around fold hinges Defaults to False.**

Text(value='False', description='use_fat')

**clut_path: User defined look up table for formation colours Default ''.**

Text(value='', description='clut_path')

In [74]:
aus=aus_choice.value
deposits=deposits_choice.value
dtb=dtb.value
orientation_decimate=orientation_decimate.value
contact_decimate=contact_decimate.value
intrusion_mode=intrusion_mode.value
interpolation_spacing=interpolation_spacing.value
misorientation=misorientation.value
interpolation_scheme=interpolation_scheme.value
fault_decimate=fault_decimate.value
min_fault_length=min_fault_length.value
fault_dip=fault_dip.value
pluton_dip=pluton_dip.value
pluton_form=pluton_form.value
dist_buffer=dist_buffer.value
contact_dip=contact_dip.value
contact_orientation_decimate=contact_orientation_decimate.value
null_scheme=null_scheme.value
thickness_buffer=thickness_buffer.value
max_thickness_allowed=max_thickness_allowed.value
fold_decimate=fold_decimate.value
fat_step=fat_step.value
close_dip=close_dip.value
use_interpolations=use_interpolations.value
use_fat=use_fat.value
clut_path=clut_path.value


In [80]:
f=open(test_data_name+'/run_m2l.py',"w")
f.write("\
\
import os\n\
from map2loop.project import Project\n\
\n\
proj = Project(\n\
                  geology_file='"+geology_file+"',\n\
                  fault_file='"+fault_file+"',\n\
                  fold_file='"+fold_file+"',\n\
                  structure_file='"+structure_file+"',\n\
                  mindep_file='"+mindep_file+"',\n\
                  metadata='./"+test_data_name+"_config.hjson'\n\
                )\n\
\n\
proj.update_config(\n\
                    out_dir='"+test_data_name+"',\n\
                    overwrite='true',\n\
                    bbox_3d={\n\
                        'minx': "+minx+" ,\n\
                        'miny': "+miny+" ,\n\
                        'maxx': "+maxx+" ,\n\
                        'maxy': "+maxy+" ,\n\
                        'base': "+minz+",\n\
                        'top':  "+maxz+",\n\
                        'local' : True,\n\
                        'aus': "+aus+",\n\
                        'close_dip': "+close_dip+",\n\
                        'contact_decimate': "+contact_decimate+",\n\
                        'contact_dip': "+contact_dip+",\n\
                        'contact_orientation_decimate': "+contact_orientation_decimate+",\n\
                        'deposits': "+deposits+",\n\
                        'dist_buffer': "+dist_buffer+",\n\
                        'dtb': '"+dtb+"',\n\
                        'fat_step': "+fat_step+",\n\
                        'fault_decimate': "+fault_decimate+",\n\
                        'fault_dip': "+fault_dip+",\n\
                        'fold_decimate': "+fold_decimate+",\n\
                        'interpolation_scheme': '"+interpolation_scheme+"',\n\
                        'interpolation_spacing': "+interpolation_spacing+",\n\
                        'intrusion_mode': "+intrusion_mode+",\n\
                        'max_thickness_allowed': "+max_thickness_allowed+",\n\
                        'min_fault_length': "+min_fault_length+",\n\
                        'misorientation': "+misorientation+",\n\
                        'null_scheme': '"+null_scheme+"',\n\
                        'orientation_decimate': "+orientation_decimate+",\n\
                        'pluton_dip': "+pluton_dip+",\n\
                        'pluton_form': '"+pluton_form+"',\n\
                        'thickness_buffer': "+thickness_buffer+",\n\
                        'use_fat': "+use_fat+",\n\
                        'use_interpolations': "+use_interpolations+",\n\
                        'clut_path': '"+clut_path+"',\n\
                     },\n\
                    proj_crs={'init': '"+dst_crs+"'},\n\
                    quiet=True\n\
                  )\n\
\nproj.run()\n"   
        
)
f.write("\n")
f.close()

In [65]:
aus

Text(value='True', description='aus')

In [66]:
dst_crs='epsg:'+epsg_code[0]

In [67]:
dst_crs

'epsg:28351'